In [3]:
import os
import sys
import codecs

In [4]:
path_to_alignments = '/usr0/home/glample/Research/IncrementalMT/en-zh.tok.sym.grow-diag'
path_to_bitext = '/usr0/home/glample/Research/IncrementalMT/en-zh.tok'

In [13]:
alignments = [line.strip().split() for line in open(path_to_alignments, 'r')][900003:]
bitext = [line.strip().split(' ||| ') for line in codecs.open(path_to_bitext, 'r', encoding='utf8')][900003:]

In [14]:
assert len(alignments) == len(bitext)

In [21]:
english_sents = [x[0].split() for x in bitext]
french_sents = [x[1].split() for x in bitext]

In [22]:
def parse_alignment(alignment):
    forward_alignment = {}
    backward_alignment = {}
    for alignment_pair in alignment:
        alignment_pair = [int(x) for x in alignment_pair.split('-')]
        if alignment_pair[0] not in forward_alignment:
            forward_alignment[alignment_pair[0]] = [alignment_pair[1]]
        else:
            forward_alignment[alignment_pair[0]].append(alignment_pair[1])
        if alignment_pair[1] not in backward_alignment:
            backward_alignment[alignment_pair[1]] = [alignment_pair[0]]
        else:
            backward_alignment[alignment_pair[1]].append(alignment_pair[0])
    return forward_alignment, backward_alignment

In [23]:
def get_not_in_target(alignment):
    targets = [int(x.split('-')[0]) for x in alignment]
    return set(range(max(targets))) - set(targets)

In [24]:
#print '==============================================================================================='
actions = []
for ind, (alignment, english_sent, french_sent) in enumerate(zip(alignments, english_sents, french_sents)):
    
    if ind % 1000 == 0:
        print 'Finished %d out of %d ' %(ind, len(english_sents))
    indices = set()
    backward_alignment, forward_alignment = parse_alignment(alignment)
    not_in_target = get_not_in_target(alignment)
    target_start = 0
    source_start = 0
    actionset = []
    
    for ind, word in enumerate(english_sent):
        
        # If word in source doesn't have an alignment, SHIFT
        if ind not in forward_alignment:
            #print u'{: <2} | {: <20} | {: <2} | {: <30} | {: <30} '.format(ind, word, 'NS', ' '.join(english_sent[source_start:ind+1]), 'NULL')
            actionset.append([word, 'S', ' '.join(english_sent[source_start:ind+1]), ' '.join(english_sent[ind+1:]),  '$NONE$'])
            continue
        
        # Add all the alignments for this word to indices
        for item in forward_alignment[ind]:
            indices.add(item)
        action = 'T'
        max_ind = max(indices)
        
        # Check if all target indices of the max spanning source block alignment are contained if not, SHIFT
        for i in range(target_start, max_ind):
            if i in not_in_target:
                continue
            if i not in indices:
                action = 'S'
                break
        
        # Write the SHIFT action to the actionset
        if action == 'S':
            #print u'{: <2} | {: <20} | {: <2} | {: <30} | {: <30} | {: <10}  '.format(ind, word, action, ' '.join(english_sent[source_start:ind+1]), 'NULL', str(target_start) + ', ' + str(max_ind) + ' -> ' + ' '.join([str(x) for x in indices]))
            actionset.append([word, action, ' '.join(english_sent[source_start:ind+1]), ' '.join(english_sent[ind+1:]), '$NONE$'])
        
        # Write the TRANSLATE action to the actionset
        elif action == 'T':
            translation = french_sent[target_start:max_ind+1]
            curr_phrase_block = english_sent[source_start:ind+1]
            #print u'{: <2} | {: <20} | {: <2} | {: <30} | {: <30} | {: <10} '.format(ind, word, action, ' '.join(curr_phrase_block), ' '.join(translation), str(target_start)  + ' -> ' + str(max_ind))
            actionset.append([word, action, ' '.join(curr_phrase_block), ' '.join(english_sent[ind+1:]), ' '.join(translation) if translation != [] else '$NONE$'])
            target_start = max(max_ind + 1, target_start) # Max to handle the case where a word in the source maps backwards in the target
            source_start = ind + 1
            indices = set()
    actionset.insert(0, ' '.join(french_sent))
    actionset.insert(0, ' '.join(english_sent))
    #print ' '.join(french_sent)
    #print '==============================================================================================='
    actions.append(actionset)

Finished 0 out of 45020 
Finished 1000 out of 45020 
Finished 2000 out of 45020 
Finished 3000 out of 45020 
Finished 4000 out of 45020 
Finished 5000 out of 45020 
Finished 6000 out of 45020 
Finished 7000 out of 45020 
Finished 8000 out of 45020 
Finished 9000 out of 45020 
Finished 10000 out of 45020 
Finished 11000 out of 45020 
Finished 12000 out of 45020 
Finished 13000 out of 45020 
Finished 14000 out of 45020 
Finished 15000 out of 45020 
Finished 16000 out of 45020 
Finished 17000 out of 45020 
Finished 18000 out of 45020 
Finished 19000 out of 45020 
Finished 20000 out of 45020 
Finished 21000 out of 45020 
Finished 22000 out of 45020 
Finished 23000 out of 45020 
Finished 24000 out of 45020 
Finished 25000 out of 45020 
Finished 26000 out of 45020 
Finished 27000 out of 45020 
Finished 28000 out of 45020 
Finished 29000 out of 45020 
Finished 30000 out of 45020 
Finished 31000 out of 45020 
Finished 32000 out of 45020 
Finished 33000 out of 45020 
Finished 34000 out of 45020

In [25]:
f = codecs.open('oracle.en.zh.tsv', 'w', encoding='utf8')
for actionset in actions:
    f.write(actionset[0] + '\n')
    f.write(actionset[1] + '\n')
    for action in actionset[2:]:
        f.write(' ||| '.join(action) + '\n')
    f.write('\n')
f.close()